In [27]:
#!pip install pandas plotly scikit-learn optuna ipywidgets ipykernel nbformat gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(suppress=True, precision=10)

#Machine Learning
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

import optuna

### Carregamento dos dados

In [ ]:
#Carregar dados
df_clientes = pd.read_csv('dataset.csv')

In [4]:
#Visualização do Dataframe
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [5]:
#Visualização dos primeiros registros
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


In [6]:
#Visualização dos ultimos registros
df_clientes.tail(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
490,Indústria,215580.61,11,Belo Horizonte,7,3
491,Serviços,1050776.57,14,Vitória,8,0
492,Comércio,785671.05,15,Vitória,9,2
493,Serviços,658330.45,20,Belo Horizonte,4,8
494,Agronegócio,1643153.26,14,Rio de Janeiro,10,1
495,Serviços,1581841.42,17,Rio de Janeiro,8,2
496,Indústria,1291309.57,9,São Paulo,6,9
497,Serviços,2211489.85,10,Belo Horizonte,10,0
498,Agronegócio,1460860.46,12,Rio de Janeiro,5,3
499,Indústria,173684.43,13,Belo Horizonte,4,9


In [7]:
#Medidas estatísticas
df_clientes.describe()

,faturamento_mensal,numero_de_funcionarios,idade,inovacao
count,500.00,500.00,500.00,500.00
mean,1026715.63,13.69,9.25,4.39
std,420609.46,3.12,2.96,2.90
min,18421.22,2.00,0.00,0.00
25%,763253.58,12.00,7.00,2.00
50%,1022957.08,14.00,9.00,4.00
75%,1295888.52,16.00,11.00,7.00
max,2390677.22,21.00,16.00,9.00


### Preparar dados para execução do modelo GMM

In [8]:
#Seleção de colunas relevantes para a clusterização
X = df_clientes.copy()

#Separação das variáveis por tipo para aplicar o ColumnTransformer
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

#Criar as transformações a serem aplicadas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

#Criar o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ]
)

#Transformar os dados
X_transformed = preprocessor.fit_transform(X)

In [9]:
#Visualização X_transformed
X_transformed

array([[-0.7463449774, -0.5417919104, -1.1005884861, ...,  0.          ,
         0.          ,  1.          ],
       [-0.5616554761, -1.5035526981,  1.9434485069, ...,  0.          ,
         0.          ,  0.          ],
       [ 0.4058265391,  1.0611427358, -1.7770411512, ...,  0.          ,
         0.          ,  9.          ],
       ...,
       [ 2.8196246022, -1.1829657689,  0.2523168441, ...,  0.          ,
         1.          ,  0.          ],
       [ 1.0332141129, -0.5417919104, -1.4388148187, ...,  0.          ,
         0.          ,  3.          ],
       [-2.0301148644, -0.2212049812, -1.7770411512, ...,  1.          ,
         0.          ,  9.          ]])

### Treinamento do modelo GMM

In [10]:
#Criação de função para executar no Optuna
def gmm_objective(trial):
  #Definindo os hiperparâmetros a serem ajustados
  n_components = trial.suggest_int('n_components', 3, 10)
  covariance_type = trial.suggest_categorical('covariance_type', ['full', 'tied', 'diag', 'spherical'])

  #Instanciar o modelo GMM com os hiperparâmetros
  gmm = GaussianMixture(n_components=n_components, covariance_type=covariance_type, random_state=51)

  #Treinar o modelo nos dados
  gmm.fit(X_transformed)

  #Calculando o BIC(Bayesian Information Criterial)
  bic_gmm = gmm.bic(X_transformed)

  return bic_gmm

In [11]:
#Criar um estudo do Optuna
search_space = {'n_components': [3, 4, 5, 6, 7, 8, 9, 10],
                'covariance_type': ['full', 'tied', 'diag', 'spherical']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_gmm = optuna.create_study(direction='maximize', sampler=sampler)

[I 2025-02-24 19:23:08,600] A new study created in memory with name: no-name-9b92b2ee-170d-4e98-8bbc-8f34a4216567


In [12]:
#Execução do optuna para otimizar os hiperparâmetros
estudo_gmm.optimize(gmm_objective, n_trials=32)

[I 2025-02-24 19:23:08,698] Trial 0 finished with value: -177.47638660720793 and parameters: {'n_components': 6, 'covariance_type': 'tied'}. Best is trial 0 with value: -177.47638660720793.
[I 2025-02-24 19:23:08,730] Trial 1 finished with value: -23479.73180905732 and parameters: {'n_components': 9, 'covariance_type': 'diag'}. Best is trial 0 with value: -177.47638660720793.
[I 2025-02-24 19:23:08,759] Trial 2 finished with value: -239.16103283522068 and parameters: {'n_components': 5, 'covariance_type': 'tied'}. Best is trial 0 with value: -177.47638660720793.
[I 2025-02-24 19:23:08,873] Trial 3 finished with value: -16669.227646550302 and parameters: {'n_components': 5, 'covariance_type': 'full'}. Best is trial 0 with value: -177.47638660720793.
[I 2025-02-24 19:23:08,888] Trial 4 finished with value: 1570.0096069319204 and parameters: {'n_components': 3, 'covariance_type': 'diag'}. Best is trial 4 with value: 1570.0096069319204.
[I 2025-02-24 19:23:08,999] Trial 5 finished with val

In [13]:
#Melhor configuração obtida pelo optuna
best_params = estudo_gmm.best_params

In [14]:
#Instanciar e treinar o modelo com os melhores parâmetros do optuna
best_gmm = GaussianMixture(n_components=best_params['n_components'], covariance_type=best_params['covariance_type'], random_state=51)
best_gmm.fit(X_transformed)

#Calcular o BIC do melhor modelo
best_bic = best_gmm.bic(X_transformed)

In [15]:
#Melhores parâmetros e resultados
print("Quantidade ideal de componentes:", best_params['n_components'])
print("Tipo de Covariância:", best_params['covariance_type'])
print("BIC do melhor modelo:", best_bic)

Quantidade ideal de componentes: 3
Tipo de Covariância: spherical
BIC do melhor modelo: 13517.95476525887


### Retornando Clusters do melhor modelo GMM

In [16]:
#Execução de um predict para retornar os clusters
clusters_gmm = best_gmm.predict(X_transformed)

In [17]:
#Visualizar resultados dos clusters
clusters_gmm


array([0, 0, 1, 0, 1, 2, 0, 1, 0, 2, 2, 2, 1, 2, 1, 1, 1, 0, 1, 2, 0, 0,
       2, 2, 0, 0, 1, 1, 0, 0, 0, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2,
       0, 1, 2, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 2, 1, 1, 2, 0, 1, 1, 2,
       2, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 2, 2, 1, 1, 0, 0, 1, 2, 1, 1, 0,
       0, 1, 0, 2, 2, 0, 0, 1, 1, 1, 1, 2, 0, 0, 2, 2, 0, 2, 1, 2, 2, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 2, 2, 0, 0, 2, 2, 1, 0, 1, 2, 2, 1, 1, 0,
       0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 2, 2, 1,
       0, 1, 1, 1, 0, 0, 2, 1, 1, 1, 2, 2, 0, 0, 1, 1, 1, 1, 0, 2, 1, 0,
       1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 1, 0, 1, 1, 1, 0, 2, 1, 0, 1, 0, 0,
       2, 2, 2, 1, 1, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0,
       1, 2, 1, 1, 1, 1, 0, 1, 0, 2, 0, 2, 1, 1, 0, 1, 2, 2, 1, 2, 1, 2,
       0, 0, 0, 1, 2, 1, 1, 1, 1, 0, 0, 0, 1, 2, 1, 1, 1, 0, 2, 1, 0, 1,
       0, 2, 1, 1, 0, 2, 1, 1, 0, 1, 1, 2, 2, 1, 2, 1, 2, 1, 0, 0, 1, 1,
       1, 1, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 0, 2,

In [18]:
#Gerar as probabilidades de cada registro estar em um cluster
clusters_gmm_prob = best_gmm.predict_proba(X_transformed)

In [19]:
#Mostrar as probabilidades
clusters_gmm_prob

array([[0.9999798986, 0.          , 0.0000201014],
       [0.9999999957, 0.          , 0.0000000043],
       [0.          , 1.          , 0.          ],
       ...,
       [0.9999999977, 0.          , 0.0000000023],
       [0.053478558 , 0.0000000173, 0.9465214247],
       [0.          , 1.          , 0.          ]])

In [20]:
#Criar uma coluna chamada cluster para armazenar o cluster do gmm
df_clientes['cluster'] = clusters_gmm.astype(int)

In [21]:
#Visualização do Dataframe atualizado
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


### Visualização dos resultados

In [22]:
#Cruzar dados de idade, faturamento mensal e clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='cluster')

In [23]:
#Cruzar dados de inovação, faturamento mensal e clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='cluster')

In [24]:
#Cruzar dados de inovação, idade e clusters
px.scatter(df_clientes, x='inovacao', y='idade', color='cluster')

### Salvar modelo e Pipeline de transformação

In [25]:
import joblib

#Salvar modelo
joblib.dump(best_gmm, 'modelo_clusterizacao_clientes_gmm.pkl')

#Salvar pipeline
joblib.dump(preprocessor, 'pipeline_clusterizacao_clientes_gmm.pkl')

['pipeline_clusterizacao_clientes_gmm.pkl']

### Aplicação de geração dos clusters
- Lê arquivo
- Aplica modelo
- Gera arquivo com cluster

In [28]:
import gradio as gr

modelo = joblib.load('./modelo_clusterizacao_clientes_gmm.pkl')
pipeline = joblib.load('./pipeline_clusterizacao_clientes_gmm.pkl')

In [29]:
def clustering(arquivo):
  df_empresas = pd.read_csv(arquivo.name)

  #Transformar os dados do Dataframe para o modelo
  X_transformed = pipeline.fit_transform(df_empresas)

  #Treinamento do modelo gmm
  modelo.fit(X_transformed)

  #Fazer predições
  clusters_empresas = modelo.predict(X_transformed)

  #Criar coluna no Dataframe com o cluster
  df_empresas['cluster'] = clusters_empresas.astype(int)

  #Salvar Dataframe para um arquivo
  df_empresas.to_csv('./clusters_empresas.csv', index=False)
  return './clusters_empresas.csv'

In [30]:
#Executar frontend no gradio
app = gr.Interface(
    clustering,
    gr.File(file_types=[".csv"]),
    "file"
)

app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e98e5723ca217490a1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
